## Siamese NN

In [2]:
import os
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
from SNN.siamese_network import *
from SNN.siamese_network_parse import PrepareDataSNN
from sklearn.model_selection import train_test_split

## Preparar la data

In [3]:
data = PrepareDataSNN()

# Ruta para guardar los datos
data_dir = "./data_snn"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

data_file = os.path.join(data_dir, "dataset_split.npz")

if os.path.exists(data_file):
    # Cargar los datos si ya están guardados
    print("Cargando conjuntos de entrenamiento y validación desde archivo...")
    data = np.load(data_file)
    X1_train, X1_val = data["X1_train"], data["X1_val"]
    X2_train, X2_val = data["X2_train"], data["X2_val"]
    y_train, y_val = data["y_train"], data["y_val"]
else:
    # Crear los conjuntos de datos y guardarlos
    print("Dividiendo los datos y guardándolos...")
    data_a, data_b, labels, names = PrepareDataSNN().process()
    X1_train, X1_val, X2_train, X2_val, y_train, y_val = train_test_split(
        data_a, data_b, labels, test_size=0.2, random_state=42
    )
    np.savez(data_file, X1_train=X1_train, X1_val=X1_val, 
                X2_train=X2_train, X2_val=X2_val, 
                y_train=y_train, y_val=y_val)

# Verificar las formas de los arrays
print("Formas de los conjuntos de entrenamiento:")
print(f"X1_train: {X1_train.shape}")
print(f"X2_train: {X2_train.shape}")
print(f"y_train: {y_train.shape}")

print("\nFormas de los conjuntos de validación:")
print(f"X1_val: {X1_val.shape}")
print(f"X2_val: {X2_val.shape}")
print(f"y_val: {y_val.shape}")


Cargando conjuntos de entrenamiento y validación desde archivo...
Formas de los conjuntos de entrenamiento:
X1_train: (726, 65)
X2_train: (726, 65)
y_train: (726,)

Formas de los conjuntos de validación:
X1_val: (182, 65)
X2_val: (182, 65)
y_val: (182,)


## Inicializar la red siamesa, entrenar y guardar el modelo

In [4]:
# Inicializar y entrenar la red
input_shape = (65,)  # Asumiendo que tus vectores de características tienen 65 elementos
siamese_net = SiameseNeuralNetwork(input_shape)
history = siamese_net.train(X1_train, X2_train, y_train, validation_data=(X1_val, X2_val, y_val))

# Guardar el modelo
siamese_net._save_model("model_1")

Epoch 1/20


/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.4611 - loss: 1.9649 - val_accuracy: 0.5110 - val_loss: 1.9563
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4838 - loss: 1.6423 - val_accuracy: 0.4945 - val_loss: 1.8916
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4936 - loss: 1.6240 - val_accuracy: 0.5110 - val_loss: 1.7826
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4880 - loss: 1.5701 - val_accuracy: 0.5055 - val_loss: 1.7762
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4921 - loss: 1.5530 - val_accuracy: 0.4945 - val_loss: 1.6845
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4933 - loss: 1.5281 - val_accuracy: 0.4945 - val_loss: 1.6454
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4936 - loss: 1.5197 - val_accuracy: 0.5000 - val_loss: 1.6471
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4846 - loss: 1.4743 - val_accuracy: 0.5055 - val_loss: 1.6174
Ep

## Cargar el modelo guardado

In [5]:
model = load_model('./models/model_1.keras', custom_objects={'L1Distance': L1Distance})

model.summary()

TypeError: Could not locate function 'asymmetric_loss'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'builtins', 'class_name': 'function', 'config': 'asymmetric_loss', 'registered_name': 'function'}

## Preparar la nueva data de validacion (Proyectos de C#)

In [6]:

# Ruta para guardar los datos
data_dir = "./data_val"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

data_file = os.path.join(data_dir, "dataset_split.npz")

if os.path.exists(data_file):
    # Cargar los datos si ya están guardados
    print("Cargando conjuntos de validación desde archivo...")
    data = np.load(data_file)
    X1_new, X2_new, y_new, names = data["X1_new"], data["X2_new"], data["y_new"], data["names"]
else:
    print("Procesando los datos de validación y guardándolos...")
    data_val = PrepareDataSNN("data/features_vect_val/")
    X1_new, X2_new, y_new, names = data_val.process()
    
    # Guardar los datos procesados
    np.savez(data_file, X1_new=X1_new, X2_new=X2_new, y_new=y_new, names=names)


# Verificar las formas de los arrays
print("Formas de los conjuntos de validación:")
print(f"X1_new: {X1_new.shape}")
print(f"X2_new: {X2_new.shape}")
print(f"y_new: {y_new.shape}")


Cargando conjuntos de validación desde archivo...
Formas de los conjuntos de validación:
X1_new: (140, 65)
X2_new: (140, 65)
y_new: (140,)


## Evaluar el modelo con los datos nuevos

In [7]:
m = model.evaluate([X1_new, X2_new], y_new)

predictions = (model.predict([X1_new, X2_new]) > 0.5).astype(int).flatten()
        
# Calcular la matriz de confusión
tn, fp, fn, tp = confusion_matrix(y_new, predictions).ravel()

# Calcular métricas adicionales
precision = precision_score(y_new, predictions)
recall = recall_score(y_new, predictions)
f1 = f1_score(y_new, predictions)

# Imprimir las métricas adicionales
print(f"Verdaderos Positivos (TP): {tp}")
print(f"Falsos Positivos (FP): {fp}")
print(f"Verdaderos Negativos (TN): {tn}")
print(f"Falsos Negativos (FN): {fn}")
print(f"Precisión (Precision): {precision:.4f}")
print(f"Exhaustividad (Recall): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

NameError: name 'model' is not defined

In [8]:
metrics = siamese_net.evaluate(X1_new, X2_new, y_new, names)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7465 - loss: 0.9037 
Pérdida en los nuevos datos: 0.8873
Accuracy en los nuevos datos: 0.7500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Verdaderos Positivos (TP): 27
Falsos Positivos (FP): 15
Verdaderos Negativos (TN): 90
Falsos Negativos (FN): 8
Precisión (Precision): 0.6429
Exhaustividad (Recall): 0.7714
F1-Score: 0.7013

Falsos negativos:
Par: ['HulkCompiler' 'copy_of_HulkCompiler']
Par: ['Pol472_MoogleX' 'copy_of_Pol472_MoogleX']
Par: ['Gelita04_moogle' 'copy_of_Gelita04_moogle']
Par: ['copy_of_moogle-main-Bellmehn' 'moogle-main-Bellmehn']
Par: ['HulkCompiler' 'copy_of_HulkCompiler']
Par: ['Dominoes-main' 'copy_of_Dominoes-main']
Par: ['copy_of_Moogle_Abraham-master' 'Moogle_Abraham-master']
Par: ['Moist-Cat_moogle-2023' 'copy_of_Moist-Cat_moogle-2023']

Verdaderos positivos:
Par: ['copy_of_LoLProM_Moogle' 'LoLProM_Moogle']
Par: ['Javi111003_moogle' 'copy_of_Javi111003_moogle']
Par: ['HULK' 'copy_of_HULK']
Par: ['copy_of_Srdariansitop_

## Hacer una predicción

In [8]:
code1_features = np.random.rand(65)  # Ejemplo de vector de características
code2_features = np.random.rand(65)  # Ejemplo de vector de características

similarity = siamese_net.predict_similarity(code1_features, code2_features)

# Interpretar el resultado
if similarity < 0.5:
    print(f"Los códigos son similares con una probabilidad de {1-similarity:.2f}")
else:
    print(f"Los códigos son diferentes con una probabilidad de {similarity:.2f}")

print(f"La similitud entre los dos códigos es: {similarity}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Los códigos son similares con una probabilidad de 0.52
La similitud entre los dos códigos es: 0.48060810565948486


## Buscando la mejor semilla para el modelo

In [10]:
import os
import tensorflow as tf
import numpy as np
import random

# Definir un rango de semillas
SEEDS = range(50)  # Probar con semillas del 0 al 49

# Variables para almacenar los mejores resultados
best_accuracy = 0
best_precision = 0
best_seed = None
best_model = None

for seed in SEEDS:
    print(f"Probando con semilla: {seed}")
    
    # Configurar la semilla
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
    # Crear y entrenar el modelo
    snn = SiameseNeuralNetwork(input_shape=(65,), learning_rate=0.001, SEED=seed)
    snn.train(X1_train, X2_train, y_train, validation_data=(X1_val, X2_val, y_val), epochs=10, batch_size=32)
    
    # Evaluar el modelo
    metrics = snn.evaluate(X1_new, X2_new, y_new)
    accuracy = metrics["accuracy"]
    precision = metrics["precision"]
    
    # Comparar con los mejores resultados
    if  ( precision > best_precision):
        best_accuracy = accuracy
        best_precision = precision
        best_seed = seed
        best_model = snn.model  # Guardar el modelo actual como el mejor

# Imprimir los resultados
print(f"Mejor semilla: {best_seed}")
print(f"Mejor accuracy: {best_accuracy:.4f}")
print(f"Mejor precisión: {best_precision:.4f}")

# Guardar el mejor modelo
tf.keras.config.enable_unsafe_deserialization()
best_model.save("models/best_model_all_dataset_val.keras")


Probando con semilla: 0
Epoch 1/10


/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5104 - loss: 3.2627 - val_accuracy: 0.3352 - val_loss: 2.0639
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4541 - loss: 1.6932 - val_accuracy: 0.3846 - val_loss: 1.9361
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4594 - loss: 1.6563 - val_accuracy: 0.4835 - val_loss: 1.8644
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4953 - loss: 1.6115 - val_accuracy: 0.4945 - val_loss: 1.7515
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4986 - loss: 1.6006 - val_accuracy: 0.5055 - val_loss: 1.6609
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4933 - loss: 1.5499 - val_accuracy: 0.5055 - val_loss: 1.5886
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4953 - loss: 1.5457 - val_accuracy: 0.5000 - val_loss: 1.5740
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4931 - loss: 1.5318 - val_accuracy: 0.5000 - val_loss: 1.5541
Ep

/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5241 - loss: 2.7624 - val_accuracy: 0.3846 - val_loss: 1.9961
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4601 - loss: 1.7312 - val_accuracy: 0.4780 - val_loss: 1.8967
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4945 - loss: 1.7220 - val_accuracy: 0.4890 - val_loss: 1.8250
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4862 - loss: 1.6342 - val_accuracy: 0.5000 - val_loss: 1.7654
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4886 - loss: 1.6090 - val_accuracy: 0.5000 - val_loss: 1.6582
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4985 - loss: 1.6163 - val_accuracy: 0.5000 - val_loss: 1.6159
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4925 - loss: 1.5397 - val_accuracy: 0.5000 - val_loss: 1.5857
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5015 - loss: 1.5573 - val_accuracy: 0.5000 - val_loss: 1.5918
Ep

...

23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.4900 - loss: 2.2286 - val_accuracy: 0.4890 - val_loss: 1.8887
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4618 - loss: 1.6984 - val_accuracy: 0.5110 - val_loss: 1.8387
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4915 - loss: 1.6226 - val_accuracy: 0.5165 - val_loss: 1.7684
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4741 - loss: 1.5721 - val_accuracy: 0.5110 - val_loss: 1.6748
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5040 - loss: 1.6055 - val_accuracy: 0.5055 - val_loss: 1.6360
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4926 - loss: 1.5293 - val_accuracy: 0.5055 - val_loss: 1.6185
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4979 - loss: 1.5230 - val_accuracy: 0.5000 - val_loss: 1.5826
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4884 - loss: 1.5493 - val_accuracy: 0.5000 - val_loss: 1.5846
Ep

/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5059 - loss: 1.6789 - val_accuracy: 0.5000 - val_loss: 1.8903
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4981 - loss: 1.6259 - val_accuracy: 0.5055 - val_loss: 1.7984
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4956 - loss: 1.5817 - val_accuracy: 0.4945 - val_loss: 1.7229
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4978 - loss: 1.5354 - val_accuracy: 0.5000 - val_loss: 1.6872
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4917 - loss: 1.5162 - val_accuracy: 0.5000 - val_loss: 1.6261
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4893 - loss: 1.5149 - val_accuracy: 0.5000 - val_loss: 1.5876
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4921 - loss: 1.5048 - val_accuracy: 0.5000 - val_loss: 1.5533
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4890 - loss: 1.4855 - val_accuracy: 0.5000 - val_loss: 1.5532
Ep

/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5024 - loss: 2.4517 - val_accuracy: 0.4615 - val_loss: 1.8924
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4741 - loss: 1.6953 - val_accuracy: 0.5055 - val_loss: 1.7788
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4786 - loss: 1.5941 - val_accuracy: 0.5165 - val_loss: 1.7162
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4823 - loss: 1.5711 - val_accuracy: 0.5000 - val_loss: 1.6590
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4915 - loss: 1.5660 - val_accuracy: 0.5055 - val_loss: 1.6234
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4811 - loss: 1.5636 - val_accuracy: 0.5000 - val_loss: 1.5966
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4969 - loss: 1.5391 - val_accuracy: 0.5000 - val_loss: 1.5696
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4968 - loss: 1.5195 - val_accuracy: 0.5000 - val_loss: 1.5652
Ep